In [ ]:

from gensim import corpora , similarities , models
from nltk.tokenize import sent_tokenize  , word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import nltk
from nltk.corpus import stopwords
from operator import itemgetter
import re
import csv
import pprint

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
!pip install wikipedia
import wikipedia

wikipage  = wikipedia.page("Coronavirus").content
print(wikipage)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds. In humans and birds, they cause respiratory tract infections that can range from mild to lethal. Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS and COVID-19, which is causing an ongoing pandemic. In cows and pigs they cause diarrhea, while in mice they cause hepatitis and encephalomyelitis.
Coronaviruses constitute the subfamily Orthocoronavirinae, in the family Coronaviridae, order Nidovirales and realm Riboviria. They are enveloped viruses with a positive-sense single-stranded RNA genome and a nucleocapsid of helical symmetry. The genome size of coronaviruses ranges from approximately 26 to 32 kilobases, one of the largest among RNA viruses. They have characteristic

In [ ]:
# Sentence Tokenization
wiki_sent = sent_tokenize(wikipage)
print(wiki_sent)

['Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds.', 'In humans and birds, they cause respiratory tract infections that can range from mild to lethal.', 'Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS and COVID-19, which is causing an ongoing pandemic.', 'In cows and pigs they cause diarrhea, while in mice they cause hepatitis and encephalomyelitis.', 'Coronaviruses constitute the subfamily Orthocoronavirinae, in the family Coronaviridae, order Nidovirales and realm Riboviria.', 'They are enveloped viruses with a positive-sense single-stranded RNA genome and a nucleocapsid of helical symmetry.', 'The genome size of coronaviruses ranges from approximately 26 to 32 kilobases, one of the largest among RNA viruses.', 'They have characteristic club-shaped spikes that project from their surface, which in electron micro

In [ ]:
# Word Tokenization
wiki_words = []
for sent in wiki_sent:
  wiki_words.extend(word_tokenize(sent))
print(wiki_words)


['Coronaviruses', 'are', 'a', 'group', 'of', 'related', 'RNA', 'viruses', 'that', 'cause', 'diseases', 'in', 'mammals', 'and', 'birds', '.', 'In', 'humans', 'and', 'birds', ',', 'they', 'cause', 'respiratory', 'tract', 'infections', 'that', 'can', 'range', 'from', 'mild', 'to', 'lethal', '.', 'Mild', 'illnesses', 'in', 'humans', 'include', 'some', 'cases', 'of', 'the', 'common', 'cold', '(', 'which', 'is', 'also', 'caused', 'by', 'other', 'viruses', ',', 'predominantly', 'rhinoviruses', ')', ',', 'while', 'more', 'lethal', 'varieties', 'can', 'cause', 'SARS', ',', 'MERS', 'and', 'COVID-19', ',', 'which', 'is', 'causing', 'an', 'ongoing', 'pandemic', '.', 'In', 'cows', 'and', 'pigs', 'they', 'cause', 'diarrhea', ',', 'while', 'in', 'mice', 'they', 'cause', 'hepatitis', 'and', 'encephalomyelitis', '.', 'Coronaviruses', 'constitute', 'the', 'subfamily', 'Orthocoronavirinae', ',', 'in', 'the', 'family', 'Coronaviridae', ',', 'order', 'Nidovirales', 'and', 'realm', 'Riboviria', '.', 'They',

In [ ]:
# POS Tagging
tagged = pos_tag(wiki_words)
print(tagged)

[('Coronaviruses', 'NNS'), ('are', 'VBP'), ('a', 'DT'), ('group', 'NN'), ('of', 'IN'), ('related', 'JJ'), ('RNA', 'NNP'), ('viruses', 'VBZ'), ('that', 'IN'), ('cause', 'NN'), ('diseases', 'NNS'), ('in', 'IN'), ('mammals', 'NNS'), ('and', 'CC'), ('birds', 'NNS'), ('.', '.'), ('In', 'IN'), ('humans', 'NNS'), ('and', 'CC'), ('birds', 'NNS'), (',', ','), ('they', 'PRP'), ('cause', 'VBP'), ('respiratory', 'JJ'), ('tract', 'NN'), ('infections', 'NNS'), ('that', 'WDT'), ('can', 'MD'), ('range', 'VB'), ('from', 'IN'), ('mild', 'NN'), ('to', 'TO'), ('lethal', 'VB'), ('.', '.'), ('Mild', 'NNP'), ('illnesses', 'VBZ'), ('in', 'IN'), ('humans', 'NNS'), ('include', 'VBP'), ('some', 'DT'), ('cases', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('common', 'JJ'), ('cold', 'NN'), ('(', '('), ('which', 'WDT'), ('is', 'VBZ'), ('also', 'RB'), ('caused', 'VBN'), ('by', 'IN'), ('other', 'JJ'), ('viruses', 'NNS'), (',', ','), ('predominantly', 'RB'), ('rhinoviruses', 'VBZ'), (')', ')'), (',', ','), ('while', 'IN'), (

In [ ]:
from nltk.corpus import wordnet
def getpos(treebank_tag):

  if treebank_tag.startswith('J'):
    return wordnet.ADJ
  elif treebank_tag.startswith('V'):
    return wordnet.VERB
  elif treebank_tag.startswith('N'):
    return wordnet.NOUN
  elif treebank_tag.startswith('R'):
    return wordnet.ADV
  else:
    return ''

In [ ]:
# Lemmatization with pos
punctuation = u",.?!()-\"\'\\+<>#&$^|/"
stop_words_eng = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
lemma_pos = []
for word , tag in tagged:
  if word not in punctuation and word not in stop_words_eng and word.isalpha():
    p = getpos(tag)
    if p != '':
      l = wordnet_lemmatizer.lemmatize(word , pos = p)
      lemma_pos.append(l)
print(lemma_pos)

['Coronaviruses', 'group', 'related', 'RNA', 'viruses', 'cause', 'disease', 'mammal', 'bird', 'human', 'bird', 'cause', 'respiratory', 'tract', 'infection', 'range', 'mild', 'lethal', 'Mild', 'illnesses', 'human', 'include', 'case', 'common', 'cold', 'also', 'cause', 'virus', 'predominantly', 'rhinoviruses', 'lethal', 'variety', 'cause', 'SARS', 'MERS', 'cause', 'ongoing', 'pandemic', 'cow', 'pig', 'cause', 'diarrhea', 'mouse', 'cause', 'hepatitis', 'encephalomyelitis', 'Coronaviruses', 'constitute', 'subfamily', 'Orthocoronavirinae', 'family', 'Coronaviridae', 'order', 'Nidovirales', 'realm', 'Riboviria', 'enveloped', 'virus', 'RNA', 'genome', 'nucleocapsid', 'helical', 'symmetry', 'genome', 'size', 'coronaviruses', 'range', 'approximately', 'kilobases', 'large', 'RNA', 'virus', 'characteristic', 'spike', 'project', 'surface', 'electron', 'micrographs', 'create', 'image', 'reminiscent', 'solar', 'corona', 'name', 'derive', 'Etymology', 'name', 'coronavirus', 'derive', 'Latin', 'corona

In [ ]:
id2word = corpora.Dictionary([lemma_pos])
texts = lemma_pos
corpus = [id2word.doc2bow([text]) for text in texts]

n_topics = 5
lda_model = models.LdaModel(corpus = corpus , 
                            id2word = id2word,
                            num_topics = 5,
                            random_state=100,
                            update_every=1,
                            chunksize = 100,
                            passes = 10,
                            alpha  = 'symmetric',
                            per_word_topics = True                  
                            )
print(lda_model.print_topics())

[(0, '0.072*"cause" + 0.043*"pig" + 0.033*"strain" + 0.030*"bat" + 0.022*"different" + 0.019*"high" + 0.018*"bronchitis" + 0.017*"infectious" + 0.016*"mortality" + 0.016*"MHV"'), (1, '0.064*"coronaviruses" + 0.031*"PEDV" + 0.030*"disease" + 0.026*"member" + 0.025*"protein" + 0.024*"viral" + 0.024*"syndrome" + 0.022*"Coronaviruses" + 0.019*"acute" + 0.015*"Betacoronavirus"'), (2, '0.042*"also" + 0.040*"human" + 0.037*"outbreak" + 0.031*"highly" + 0.028*"reading" + 0.026*"diseases" + 0.024*"diarrhea" + 0.020*"identify" + 0.015*"target" + 0.014*"transmit"'), (3, '0.159*"coronavirus" + 0.078*"virus" + 0.056*"animal" + 0.043*"case" + 0.042*"specie" + 0.035*"use" + 0.032*"Coronavirus" + 0.032*"respiratory" + 0.029*"transmission" + 0.025*"measure"'), (4, '0.049*"herd" + 0.041*"infect" + 0.025*"References" + 0.025*"contagious" + 0.025*"Further" + 0.025*"destruction" + 0.025*"entire" + 0.025*"prevent" + 0.025*"Zoonosis" + 0.018*"laboratory"')]


In [ ]:

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, corpus, id2word)
lda_viz

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.076711 -0.145111       1        1  20.894557
0      0.068567 -0.017263       2        1  20.654137
3     -0.284363  0.000669       3        1  19.711709
4      0.065502 -0.075090       4        1  19.504306
2      0.073583  0.236795       5        1  19.235290, topic_info=              Term       Freq      Total Category  logprob  loglift
357    coronavirus  79.000000  79.000000  Default  30.0000  30.0000
940          virus  39.000000  39.000000  Default  29.0000  29.0000
302          cause  38.000000  38.000000  Default  28.0000  28.0000
358  coronaviruses  34.000000  34.000000  Default  27.0000  27.0000
250         animal  28.000000  28.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
591          large   2.500507   3.089129   Topic5  -5.2687   1.4370
688       pandemic   2.472421   3.061613   Topic5  -5.2800   1.4347
384    development   2.578722   3.193364   Topic5  -5.2379   1.4346
10       Antiviral   2.578721   3.193364   Topic5  -5.2379   1.4346
908      treatment   2.578721   3.193364   Topic5  -5.2379   1.4346

[184 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
6         1  0.967034  Alphacoronavirus
10        5  0.939448         Antiviral
23        1  0.956155   Betacoronavirus
34        2  0.920952            Canine
43        4  0.833989               CoV
...     ...       ...               ...
936       4  0.856758        veterinary
937       1  0.987589             viral
940       3  0.985943             virus
946       5  0.915212              well
957       1  0.934608             young

[154 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 5, 3])